<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/vqe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum advantage in ground state energy calculation of O₂**

* Goal: Find the lowest eigenvalues of the O2 molecular Hamiltonian.
* Method: Variational Quantum Eigensolver (VQE) for the ground state and Variational Quantum Deflation (VQD) for the excited state.
* Ansatz: AllSinglesDoubles quantum circuit to prepare the trial states for both cases

In [ ]:
!pip install pennylane openfermion openfermionpyscf -q

In [ ]:
#!/usr/bin/env python3
import pennylane as qml
from pennylane import qchem
import numpy as np
from multiprocessing import Pool
import time
from functools import partial
import warnings
from pennylane import AllSinglesDoubles
warnings.filterwarnings('ignore')

# ============================================
# 1. MOLECULE SETUP
# ============================================

def setup_molecule(active_electrons=2, active_orbitals=3, r=2.30):
    symbols = ['O', 'O']
    coordinates = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, r]])
    molecule = qchem.Molecule(symbols, coordinates)

    H, qubits = qchem.molecular_hamiltonian(
        molecule,
        active_electrons=active_electrons,
        active_orbitals=active_orbitals,
        method='openfermion'
    )

    print(f"System: {qubits} qubits, {len(H)} Hamiltonian terms")
    return H, qubits, molecule

# ============================================
# 2. SIMPLE VQE WITHOUT MULTIPROCESSING ISSUES
# ============================================

def simple_vqe(H, qubits, active_electrons, max_iterations=150):
    """Simple VQE that works without multiprocessing issues"""

    # Setup
    singles, doubles = qchem.excitations(active_electrons, qubits)
    hf_state = qchem.hf_state(active_electrons, qubits)
    n_params = len(singles) + len(doubles)

    # Create device
    dev = qml.device("lightning.qubit", wires=qubits)

    @qml.qnode(dev)
    def circuit(params):
        qml.AllSinglesDoubles(params, range(qubits), hf_state, singles, doubles)
        return qml.expval(H)

    # Initialize parameters
    params = np.random.randn(n_params) * 0.05

    # Manual gradient descent (avoiding qml.grad issues)
    learning_rate = 0.4
    best_energy = float('inf')
    best_params = params.copy()

    for step in range(max_iterations):
        # Current energy
        current_energy = float(circuit(params))

        # Finite difference gradient (more stable than qml.grad in multiprocessing)
        epsilon = 0.01
        grad = np.zeros_like(params)

        for i in range(len(params)):
            params_plus = params.copy()
            params_minus = params.copy()
            params_plus[i] += epsilon
            params_minus[i] -= epsilon

            grad[i] = (circuit(params_plus) - circuit(params_minus)) / (2 * epsilon)

        # Update parameters
        params = params - learning_rate * grad

        # Track best
        if current_energy < best_energy:
            best_energy = current_energy
            best_params = params.copy()

        # Adaptive learning rate
        if step % 40 == 0 and step > 0:
            learning_rate *= 0.8

        # Progress
        if step % 30 == 0:
            print(f"  Step {step}: Energy = {current_energy:.8f} Ha")

        # Early stopping
        if np.abs(grad).max() < 1e-5:
            break

    return best_params, best_energy

# ============================================
# 3. WRAPPER FOR PARALLEL EXECUTION
# ============================================

def run_vqe_trial(args):
    trial_num, H, qubits, active_electrons, seed = args

    # Set random seed
    np.random.seed(seed)

    print(f"  Starting trial {trial_num}...")

    try:
        best_params, best_energy = simple_vqe(H, qubits, active_electrons)
        print(f"  Trial {trial_num} completed: {best_energy:.8f} Ha")
        return trial_num, best_energy, best_params
    except Exception as e:
        print(f"  Trial {trial_num} failed: {str(e)}")
        return trial_num, float('inf'), None

# Run multiple VQE trials (can be parallel or sequential)
def run_multiple_vqe(H, qubits, active_electrons, n_trials=4):
    print(f"\nRunning {n_trials} VQE trials...")

    # Prepare arguments
    args_list = [
        (i, H, qubits, active_electrons, 42 + i * 100)
        for i in range(n_trials)
    ]

    # Try parallel execution
    try:
        with Pool(processes=min(n_trials, 96)) as pool:
            results = pool.map(run_vqe_trial, args_list)
    except:
        # Fallback to sequential if parallel fails
        print("  Parallel execution failed, running sequentially...")
        results = [run_vqe_trial(args) for args in args_list]

    # Find best result
    valid_results = [r for r in results if r[2] is not None]
    if not valid_results:
        raise ValueError("All trials failed!")

    best_result = min(valid_results, key=lambda x: x[1])
    trial_num, best_energy, best_params = best_result

    print(f"\nBest ground state energy: {best_energy:.8f} Ha (trial {trial_num})")

    return best_params, best_energy

# ============================================
# 4. EXCITED STATE VQD (SEQUENTIAL)
# ============================================

def run_vqd_excited_state(H, qubits, active_electrons, ground_params, beta=2.0):
    """Calculate excited state using VQD"""

    singles, doubles = qchem.excitations(active_electrons, qubits)
    hf_state = qchem.hf_state(active_electrons, qubits)
    n_params = len(singles) + len(doubles)

    # Device
    dev = qml.device("default.qubit", wires=qubits)


    @qml.qnode(dev)
    def energy_circuit(params):
        qml.AllSinglesDoubles(params, range(qubits), hf_state, singles, doubles)
        return qml.expval(H) # this expectation value is minimized

    @partial(qml.transforms.decompose, max_expansion=1)
    def ansatz(theta, wires):
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)#, delta_sz = 0 )
        #print(f"Total number of excitations = {len(singles) + len(doubles)}")
        #print (singles)
        singles = [[wires[i] for i in single] for single in singles]
        doubles = [[wires[i] for i in double] for double in doubles]
        #print (singles)
        AllSinglesDoubles(theta, wires, hf_state, singles, doubles)

    dev = qml.device("lightning.qubit", wires=2*qubits+1)
    @partial(qml.transforms.decompose, max_expansion=1)
    @qml.qnode(dev)
    def swap_test(params):
        # generate_ground_state(range(1, n_qubits + 1))
        #AllSinglesDoubles(ground_state_params, [1,2,3,4], hf, singles, doubles)
        ansatz(ground_params,range(1, qubits+1))
        ansatz(params, range(qubits + 1, 2 * qubits + 1))
        qml.Barrier()  # added to better visualise the circuit
        qml.Hadamard(wires=0)
        for i in range(qubits):
            qml.CSWAP(wires=[0, 1 + i + qubits, 1 + i])
        qml.Hadamard(wires=0)
        first_state_params = params
        return qml.expval(qml.Z(0))

    # if qubits <10:
    #   theta = np.random.rand(8)
    # else:
    #   theta = np.random.rand(26)

    # Overlap penalty
      # The original notebook used quantum circuit-based SWAP test to enforce orthogonality between ground and excited states
      # Here a simple classical penalty term is added where the overlap is calculated from the classical parameter vectors
      # and added to the cost function. This is less resource-intensive but also less rigorous method for ensuring states are distinct
    # def overlap_penalty(params):
    #     params_norm = params / (np.linalg.norm(params) + 1e-8)
    #     ground_norm = ground_params / (np.linalg.norm(ground_params) + 1e-8)
    #     overlap = abs(np.dot(params_norm, ground_norm))
    #     return overlap

    # Initialize orthogonal to ground state
    params = np.random.randn(n_params) * 0.1
    params = params - np.dot(params, ground_params) * ground_params / (np.linalg.norm(ground_params)**2 + 1e-8)
    params = params * 0.1

    # Optimization
    learning_rate = 0.2
    print("\nOptimizing excited state..")

    for step in range(100):
        # Current values
        energy = float(energy_circuit(params))
        overlap = swap_test(params)
        cost = energy + beta * overlap

        # Gradient using finite differences
        epsilon = 0.01
        grad = np.zeros_like(params)

        for i in range(len(params)):
            params_plus = params.copy()
            params_minus = params.copy()
            params_plus[i] += epsilon
            params_minus[i] -= epsilon

            energy_plus = energy_circuit(params_plus)
            energy_minus = energy_circuit(params_minus)
            overlap_plus = swap_test(params_plus)
            overlap_minus = swap_test(params_minus)

            cost_plus = energy_plus + beta * overlap_plus
            cost_minus = energy_minus + beta * overlap_minus

            grad[i] = (cost_plus - cost_minus) / (2 * epsilon)

        # Update
        params = params - learning_rate * grad

        if step % 20 == 0:
            print(f"  Step {step}: Energy = {energy:.6f} Ha, Overlap = {overlap:.4f}")

        # Adaptive learning rate
        if step % 40 == 0 and step > 0:
            learning_rate *= 0.8

    final_energy = float(energy_circuit(params))
    return params, final_energy

# ============================================
# 5. ANALYSIS FUNCTIONS
# ============================================

def verify_spin_state(params, qubits, active_electrons):
    """Verify spin state"""
    singles, doubles = qchem.excitations(active_electrons, qubits)
    hf_state = qchem.hf_state(active_electrons, qubits)

    S2 = qchem.spin2(active_electrons, qubits)
    dev = qml.device("lightning.qubit", wires=qubits)

    @qml.qnode(dev)
    def spin_circuit(params):
        qml.AllSinglesDoubles(params, range(qubits), hf_state, singles, doubles)
        return qml.expval(S2)

    return float(spin_circuit(params))

def compare_with_exact(H):
    """Compare with exact diagonalization"""
    eigenvalues = np.sort(np.linalg.eigvals(H.matrix()).real)
    print("\nExact eigenvalues (first 10):")
    for i, e in enumerate(eigenvalues[:10]):
        print(f"  State {i}: {e:.8f} Ha")
    return eigenvalues

# ============================================
# 6. MAIN WORKLFOW
# ============================================

def main():
    print("="*60)
    print("SIMPLE PARALLEL VQD FOR O2 MOLECULE")
    print("="*60)

    # Test different system sizes
    configs = [
        (2, 3),  # 6 qubits
        (4, 4),  # 8 qub  its
        (6, 5),  # 10 qubits
    ]

    for active_e, active_o in configs:
        print(f"\n{'='*60}")
        print(f"System: {active_e} electrons, {active_o} orbitals")
        print(f"{'='*60}")

        try:
            # Setup molecule
            H, qubits, molecule = setup_molecule(active_e, active_o)

            # Ground state
            start_time = time.time()
            ground_params, ground_energy = run_multiple_vqe(
                H, qubits, active_e, n_trials=8  # Reduced for stability
            )
            ground_time = time.time() - start_time

            # Verify spin
            s2_value = verify_spin_state(ground_params, qubits, active_e)
            print(f"\nGround state S² = {s2_value:.6f}")
            print(f"Time taken: {ground_time:.2f} seconds")

            # Excited state (only for small systems)
            if qubits <= 10:
                start_time = time.time()
                excited_params, excited_energy = run_vqd_excited_state(
                    H, qubits, active_e, ground_params
                )
                excited_time = time.time() - start_time

                print(f"\nFirst excited state energy: {excited_energy:.8f} Ha")
                print(f"Energy gap: {(excited_energy - ground_energy)*27.2114:.4f} eV")
                print(f"Time taken: {excited_time:.2f} seconds")

            # Compare with exact (only for small systems)
            if qubits <= 10:
                exact_eigenvalues = compare_with_exact(H)
                print(f"\nError vs exact:")
                print(f"  Ground state: {abs(ground_energy - exact_eigenvalues[0])*1000:.3f} mHa")

        except Exception as e:
            print(f"Error: {str(e)}")
            import traceback
            traceback.print_exc()
            continue

# ============================================
# 7. FALLBACK: SEQUENTIAL VERSION (STABLE)
# ============================================

def simple_sequential_vqd(active_electrons=2, active_orbitals=3):
    """Simplest version without any parallelization"""
    print("\n" + "="*60)
    print("Simple Sequential VQD (Most Stable)")
    print("="*60)

    # Setup
    H, qubits, molecule = setup_molecule(active_electrons, active_orbitals)

    # Ground state
    print("\nCalculating ground state...")
    ground_params, ground_energy = simple_vqe(H, qubits, active_electrons)
    print(f"Ground state energy: {ground_energy:.8f} Ha")

    # Excited state
    print("\nCalculating excited state...")
    excited_params, excited_energy = run_vqd_excited_state(
        H, qubits, active_electrons, ground_params
    )
    print(f"Excited state energy: {excited_energy:.8f} Ha")
    print(f"Energy gap: {(excited_energy - ground_energy)*27.2114:.4f} eV")

    return ground_params, excited_params, ground_energy, excited_energy

# ============================================
# EXECUTION
# ============================================

if __name__ == "__main__":
    # Check available CPU cores
    import os
    n_cores = os.cpu_count()
    print(f"Available CPU cores: {n_cores}")

    # Try parallel version
    try:
        main()
    except Exception as e:
        print(f"\nParallel version failed: {str(e)}")
        print("Running simple sequential version instead...")

        # Run simple sequential version as fallback
        simple_sequential_vqd(2, 3)

Available CPU cores: 96
SIMPLE PARALLEL VQD FOR O2 MOLECULE

System: 2 electrons, 3 orbitals
System: 6 qubits, 34 Hamiltonian terms

Running 8 VQE trials...
  Starting trial 0...
  Starting trial 3...  Starting trial 4...
  Starting trial 5...
  Starting trial 6...  Starting trial 7...


  Starting trial 2...  Starting trial 1...

  Step 0: Energy = -147.55067859 Ha  Step 0: Energy = -147.54802955 Ha

  Step 0: Energy = -147.54504403 Ha  Step 0: Energy = -147.54745905 Ha
  Step 0: Energy = -147.54995017 Ha

  Step 0: Energy = -147.54119724 Ha
  Step 0: Energy = -147.55099107 Ha
  Step 0: Energy = -147.55283062 Ha
  Step 30: Energy = -147.56212668 Ha
  Step 30: Energy = -147.55838162 Ha
  Step 30: Energy = -147.56006570 Ha
  Step 30: Energy = -147.55964319 Ha
  Step 30: Energy = -147.56019765 Ha
  Step 30: Energy = -147.56167125 Ha
  Step 30: Energy = -147.56217455 Ha
  Step 30: Energy = -147.56046888 Ha
  Step 60: Energy = -147.56771560 Ha
  Step 60: Energy = -147.56758236 Ha
  Step 60

KeyboardInterrupt: 

**Quick results review**

*(tldr: pipeline runs, but results are not always optimal yet)*

1. **Accuracy**
- **6 qubits**: Ground state error = 55.824 mHa (not great)
- **8 qubits**: Ground state error = 0.071 mHa (excellent!)
- **10 qubits**: No exact comparison, but S² ≈ 0 indicates singlet (good)

$→$ *The 8-qubit system gave the best accuracy. Suggests that 4 electrons/4 orbitals is a better active space for O₂*

2. **Excited States**
- **6 qubits**: Negative energy gap (-1.32 eV) - this is wrong! The excited state has lower energy than ground state
- **8 qubits**: Positive gap (1.63 eV) - physically correct

$→$ *Overlap values (0.0464, 0.0994) show states aren't perfectly orthogonal - could use better orthogonalization*

3. **Performance**
- 6 qubits: 79 seconds
- 8 qubits: 91 seconds
- 10 qubits: 441 seconds

*The scaling looks reasonable for our quantum simulation*

**4. Potential next steps:**
* Test larger qubit count (8 qubits (4e, 4o) for O₂ gives best accuracy here)
* Increase n_trials to 8 or 12 to use all CPU cores
* Negative gap for 6 qubits suggests the optimization got stuck, check improvements in excited state calculation
* Add better orthogonalization, like full SWAP test for more accurate excited states
